In [1]:
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
from pyspark.sql.types import *
import seaborn as sns
import pandas as pd
import numpy as np
import geopy.distance
import matplotlib
import datetime
import pickle
import os
from haversine import haversine, Unit

import fastplot
%matplotlib inline

<h2>1-reading initial input data and performing some filters</h2>

In [2]:
#schema of initial data 
schema_initial_data = 'idRequest string, deviceId string, dateTime string, \
latitude double, longitude double, speedKmh Integer, heading Integer,\
accuracyDrop Integer, EngineStatus Integer, Type Integer'
#reading csv
path = os.path.abspath(os.getcwd())
df = spark.read.csv('file:///%s../../shared/data/FCD_complete/*/*'%path,sep=",", schema = schema_initial_data)


[Stage 0:==================================================>(9977 + 23) / 10000]22/10/04 08:03:16 WARN nio.NioEventLoop: Selector.select() returned prematurely 512 times in a row; rebuilding Selector io.netty.channel.nio.SelectedSelectionKeySetSelector@43877569.
22/10/04 08:03:21 WARN datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


In [5]:
#dropping duplicates
df = df.distinct()

In [3]:
df.count()

738969717

22/10/04 08:05:05 WARN server.TransportChannelHandler: Exception in connection from /10.42.0.0:45036
java.io.IOException: Connection reset by peer
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:380)
	at io.netty.buffer.PooledUnsafeDirectByteBuf.setBytes(PooledUnsafeDirectByteBuf.java:288)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1106)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:343)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:123)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:645)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:580)
	at io.netty.channel.nio.NioEventLoop.proc

In [6]:
#dropping missing values
df = df.na.drop("any")

In [7]:
#dropping points with accuracy higher than 20
df = df.filter(df['accuracyDrop'] <= 20)

In [8]:
#dateTime format
df = df.withColumn('dateTime', F.to_timestamp('dateTime'))

In [7]:
# df.count()

<strong> Filtering points which are out of our boundary of interest </strong>

In [9]:
# ============= Torino and countryside ========================================
minLat = 44.96282106687191
minLon = 7.502048016422193
maxLat = 45.19265016665321 
maxLon = 7.791812422724604
# =============================================================================
@F.udf(returnType=BooleanType())
def drop_outside_points(lat, lon):
    condition = (lat >= minLat) & (lat <= maxLat) & (lon >= minLon) & (lon <= maxLon)
    return condition

df = df.filter(drop_outside_points('latitude', 'longitude'))
df.count()

334856698

<h2>2-adding calculated segment parameters to initial data</h2>

In [10]:
schema_inital_segment = 'index Integer,idRequest string, deviceId string, dateTime timestamp, \
latitude double, longitude double, speedKmh Integer, heading Integer,\
accuracyDrop Integer, EngineStatus Integer, Type Integer, segmentDistance double,\
segmentDuration double, segmentSpeedKmH double'

@F.pandas_udf(schema_inital_segment, functionType=F.PandasUDFType.GROUPED_MAP)
def adding_segment_parameter(df):
    os.environ["ARROW_PRE_0_15_IPC_FORMAT"] = "1"
    
    df = df.sort_values(by="dateTime")
    df.reset_index(inplace=True)
    distance=[float(0)]
    duration=[float(0)]
    speed=[float(0)]
    
    for index,row in df.iterrows():
        if index == len(df)-1:
            pass
        else:
            time_difference = df.loc[index+1,'dateTime'] - df.loc[index,'dateTime']
            time_difference = time_difference.total_seconds()
            duration.append(time_difference)
            
            org = (df.loc[index,'latitude'],df.loc[index,'longitude'])
            des = (df.loc[index+1,'latitude'],df.loc[index+1,'longitude'])
            
            distance.append(float("{0:.2f}".format(geopy.distance.distance(org,des).m)))
            
            try:
                speed.append(geopy.distance.distance(org,des).m/time_difference)
            except:
                speed.append(float("NaN"))
        
    df['segmentDistance'] = distance
    df['segmentDuration'] = duration
    df['segmentSpeedKmH'] = [v*3.6 if not pd.isna(v) else float("NaN") for v in speed ]

    return df

In [11]:
df_added_parameters = df.groupby("deviceId").apply(adding_segment_parameter)
df_added_parameters = df_added_parameters.drop('index')

In [12]:
df_added_parameters.count()

Py4JJavaError: An error occurred while calling o133.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 80 in stage 9.0 failed 4 times, most recent failure: Lost task 80.3 in stage 9.0 (TID 46321, d02-bigdata.polito.it, executor 698): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/CDH-6.2.1-1.cdh6.2.1.p0.1425774/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 361, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/opt/cloudera/parcels/CDH-6.2.1-1.cdh6.2.1.p0.1425774/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile, eval_type, runner_conf)
  File "/opt/cloudera/parcels/CDH-6.2.1-1.cdh6.2.1.p0.1425774/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 163, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/opt/cloude

In [10]:
df_added_parameters2.write.csv('file:///%s/all_merged'%path, sep=",")